In [1]:
import warnings

warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# !pip install plotly
try:
    # https://stackoverflow.com/questions/57105747/modulenotfounderror-no-module-named-plotly-graph-objects/57112843
    #     import plotly.graph_objects as go
    #     import plotly.express as px
    import plotly.express as px
    import plotly.graph_objects as go
except ImportError as e:
    from plotly import graph_objs as go
    from plotly import express as px
# import plotly.express as px
# import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import datetime as dt
from datetime import timedelta
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.tsa.api import Holt, SimpleExpSmoothing, ExponentialSmoothing
# from fbprophet import Prophet
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.stattools import adfuller

# !pip install pyramid-arima
# from pyramid.arima import auto_arima
std = StandardScaler()
# pd.set_option('display.float_format', lambda x: '%.6f' % x)
# out
# filename=r"G:\file\学校\可视化\大作业\COVID-19\COVID-19-Data-master\US\County_level_summary\US_County_summary_covid19_confirmed_transpose.csv"

state_filename_base = r"G:\file\学校\可视化\大作业\COVID-19\COVID-19-Data-master\US\State_level_summary\US_State_summary_covid19_{}_trpo.xlsx"
# state_filename_base=r"COVID-19-Data-master\US\State_level_summary\US_State_summary_covid19_{}_trpo.xlsx"
# state_filename_base=r"COVID-19-Data-master/US/State_level_summary/US_State_summary_covid19_{}_trpo.xlsx"
# filename = r"G:\file\学校\可视化\大作业\COVID-19\GIS疫情地图2020全年-至今数据\【GIS点滴疫情地图·2020年01月02日-2021年01月25日】国内每天疫情统计.xlsx"
filename = r"D:\download\GIS疫情地图2020全年-至今数据\世界2021-01-25.xlsx"

# 支持中文
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号


def get_sum(type_name):
    df = pd.read_excel(state_filename_base.format(type_name))
    # pd 每一列 求和
    df_sum = df.sum()
    # print("df_sum")
    # print(df_sum)
    return df_sum


def get_3type_df():
    df_confirmed = pd.read_excel(state_filename_base.format("confirmed"))
    df_recovered = pd.read_excel(state_filename_base.format("recovered"))
    df_death = pd.read_excel(state_filename_base.format("death"))
    # df=

    df = pd.DataFrame({})
    df["confirmed"] = get_sum("confirmed")
    df["recovered"] = get_sum("recovered")
    df["death"] = get_sum("death")
    # print("df_death")
    # print(df_death)
    # print("df_death.shape")
    # print(df_death.shape)
    return df


# covid = get_3type_df()
covid=pd.read_excel(filename)
# confirmed_col = "confirmed"
# recovered_col = "recovered"
# death_col = "death"

confirmed_col = "diagnose"
recovered_col = "cure"
death_col = "death"

X = covid[[confirmed_col,recovered_col,death_col]]
# 死亡率 Mortality
# Standard Scaling since K-Means Clustering is a distance based alogrithm
# 标准缩放，因为K-均值聚类是一种基于距离的算法
X = std.fit_transform(X)

import  util
# util.elbowMethod(X)
# util.hierarchicalClusteringTest(X)
# cluster_summary=util.final_KMeans(X,2,covid)
n_clusters=4
# cluster_summary=util.final_KMeans(X,n_clusters,covid)
cluster_summary=util.final_KMeans(X,n_clusters,covid,99)
# cluster_summary=util.final_KMeans(X,n_clusters,covid,)
# cluster_summary=util.final_KMeans(X,3,covid)
# print(cluster_summary)


def get_x():
    datewise = covid
    # yy=datewise[confirmed_col]-datewise[recovered_col]-datewise[death_col]
    # print("yy")
    # print(yy)
    # print("datewise.index")
    # print(datewise.index)
    # 总的 和
    # 根据不同的 couty
    # 确诊的 - 治愈的 - 死亡的
    # 就是现在还在患病的
    # 分配  Distribution 分布
    countrywise = datewise
    # countrywise["Mortality"]=(countrywise["Deaths"]/countrywise["Confirmed"])*100
    # countrywise["Recovery"]=(countrywise["Recovered"]/countrywise["Confirmed"])*100

    countrywise["Mortality"] = (countrywise[death_col] / countrywise[confirmed_col]) * 100
    countrywise["Recovery"] = (countrywise[recovered_col] / countrywise[confirmed_col]) * 100

    # fig=px.bar(x=datewise.index,y=datewise[confirmed_col]-datewise[recovered_col]-datewise[death_col])
    # fig.update_layout(title="Distribution of Number of Active Cases 累计患病的分布(各个县)",
    #                   xaxis_title="县",yaxis_title="Number of Cases 患病的个数",)
    # # xaxis_title="Date",yaxis_title="Number of Cases",
    # fig.show()

    # 正在患病的分布(各个县)"
    # 为什么没有显示呢

    X = countrywise[["Mortality", "Recovery"]]
    # 死亡率 Mortality
    # Standard Scaling since K-Means Clustering is a distance based alogrithm
    # 标准缩放，因为K-均值聚类是一种基于距离的算法
    X = std.fit_transform(X)

# wcss = []
# sil = []
# for i in range(2, 11):
#     # 分类的个数 去尝试 每种尝试 发现 2 3 会好一些
#     # 再根据层次聚类图 我们认为 选择分成3类比较好
#     clf = KMeans(n_clusters=i, init='k-means++', random_state=42)
#     clf.fit(X)
#     labels = clf.labels_
#     centroids = clf.cluster_centers_
#     sil.append(silhouette_score(X, labels, metric='euclidean'))
#     wcss.append(clf.inertia_)

# 肘部法则
# def ElbowMethod():
#     x = np.arange(2, 11)
#     plt.figure(figsize=(10, 5))
#     plt.plot(x, wcss, marker='o')
#     plt.xlabel("Number of Clusters 集群的个数 ")
#     # 集群;群集;
#     plt.ylabel("Within Cluster Sum of Squares (WCSS) 簇内平方和")
#     # 簇内平方和（WCSS）
#     plt.title("Elbow Method 肘部法则")
#     # –Elbow Method和轮廓...
#     # 肘部法则
#     plt.show()


# countrywise["Mortality"]=(countrywise["Deaths"]/countrywise["Confirmed"])*100
# countrywise["Recovery"]=(countrywise["Recovered"]/countrywise["Confirmed"])*100

import scipy.cluster.hierarchy as sch


# 等级制度(尤指社会或组织); 统治集团; 层次体系; hierarchy
#
# 层次聚类测试
def HierarchicalClusteringTest():
    plt.figure(figsize=(20, 15))
    # dendrogram 系统树图（一种表示亲缘关系的树状图解）;
    # 连接; 联系; 链环; 连锁; 联动装置; linkage

    dendogram = sch.dendrogram(sch.linkage(X, method="ward"))
    # dendogram.

    plt.show()


# clf_final = KMeans(n_clusters=3, init='k-means++', random_state=6)
# clf_final.fit(X)
#
# # 分类
# countrywise["Clusters"] = clf_final.predict(X)
#
# cluster_summary = pd.concat([countrywise[countrywise["Clusters"] == 1].head(15),
#                              countrywise[countrywise["Clusters"] == 2].head(15),
#                              countrywise[countrywise["Clusters"] == 0].head(15)])
# cluster_summary.style.background_gradient(cmap='Reds').format("{:.2f}")
# # 背景梯度
# # plt.show()
# print("cluster_summary")
# print(cluster_summary)
# 数据显示  治愈率是0 这是
# 我们把这些州 按照治愈率和 死亡率 分成了三类
# 根据背景梯度图 显示 一类是死亡率较高 0-死亡率其次 , 2-死亡率最低





In [2]:
cluster_summary

,id,day,country,hour,name,diagnose,suspect,cure,death,time,label,datex,continents,currentdiagnose,day.1,Clusters
5,2023161,NaN,荷兰,NaN,荷兰,948933,0,11886,13540,1611574245,NaN,NaN,欧洲,923507,2021-01-25,0
6,2023162,NaN,比利时,NaN,比利时,693666,0,19239,20779,1611574245,NaN,NaN,欧洲,653648,2021-01-25,0
7,2023163,NaN,瑞典,NaN,瑞典,547166,0,4971,11005,1611574245,NaN,NaN,欧洲,531190,2021-01-25,0
10,2023166,NaN,塞尔维亚,NaN,塞尔维亚,441299,0,50730,5336,1611574245,NaN,NaN,欧洲,385233,2021-01-25,0
13,2023169,NaN,乌克兰,NaN,乌克兰,1194328,0,953297,21924,1611574245,NaN,NaN,欧洲,219107,2021-01-25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,2023178,NaN,伊朗,NaN,伊朗,1372977,0,1164794,57383,1611574245,NaN,NaN,亚洲,150800,2021-01-25,3
24,2023180,NaN,南非,NaN,南非,1412986,0,1230520,40874,1611574245,NaN,NaN,非洲,141592,2021-01-25,3
25,2023181,NaN,哥伦比亚,NaN,哥伦比亚,2013014,0,1832168,51374,1611574245,NaN,NaN,南美洲,129472,2021-01-25,3
30,2023186,NaN,土耳其,NaN,土耳其,2429605,0,2307721,25073,1611574245,NaN,NaN,亚洲,96811,2021-01-25,3


In [3]:
# cluster_summary.style.background_gradient(cmap='Reds').format("{:.2f}")

In [4]:
cluster_summary.style.background_gradient(cmap='Reds')

,id,day,country,hour,name,diagnose,suspect,cure,death,time,label,datex,continents,currentdiagnose,day.1,Clusters
5,2023161,nan,荷兰,nan,荷兰,948933,0,11886,13540,1611574245,nan,nan,欧洲,923507,2021-01-25,0
6,2023162,nan,比利时,nan,比利时,693666,0,19239,20779,1611574245,nan,nan,欧洲,653648,2021-01-25,0
7,2023163,nan,瑞典,nan,瑞典,547166,0,4971,11005,1611574245,nan,nan,欧洲,531190,2021-01-25,0
10,2023166,nan,塞尔维亚,nan,塞尔维亚,441299,0,50730,5336,1611574245,nan,nan,欧洲,385233,2021-01-25,0
13,2023169,nan,乌克兰,nan,乌克兰,1194328,0,953297,21924,1611574245,nan,nan,欧洲,219107,2021-01-25,0
16,2023172,nan,瑞士,nan,瑞士,509279,0,317600,8345,1611574245,nan,nan,欧洲,183334,2021-01-25,0
17,2023173,nan,葡萄牙,nan,葡萄牙,636190,0,456491,10469,1611574245,nan,nan,欧洲,169230,2021-01-25,0
19,2023175,nan,印度尼西亚,nan,印度尼西亚,999256,0,809488,28132,1611574245,nan,nan,亚洲,161636,2021-01-25,0
20,2023176,nan,爱尔兰,nan,爱尔兰,187554,0,25422,2970,1611574245,nan,nan,欧洲,159162,2021-01-25,0
21,2023177,nan,斯洛文尼亚,nan,斯洛文尼亚,157293,0,2027,3555,1611574245,nan,nan,欧洲,151711,2021-01-25,0


In [5]:
cluster_summary.to_csv("cluster_summary_wrold.csv")

In [6]:
cluster_summary.to_csv("cluster_summary_wrold_utf_8.csv",encoding="utf-8")

In [7]:
cluster_summary.to_csv("cluster_summary_wrold_gbk.csv",encoding="gbk")
# gbk 是正常的

PermissionError: [Errno 13] Permission denied: 'cluster_summary_wrold_gbk.csv'

In [ ]:
cluster_summary.to_csv("cluster_summary_wrold_gbk_country_all.csv",encoding="gbk")